In [3]:
#http://sawcordwell.github.io/mdp/conservation/2015/01/10/possingham1997-1/
import numpy as np
import pandas as pd
import random
from hiive.visualization import mdpviz
from time import time
import itertools
import gym
from gym.envs.toy_text.frozen_lake import generate_random_map
from gym.envs.registration import register
from gym import wrappers
from hiive.mdptoolbox import mdp
from collections import defaultdict
import sys
from collections import namedtuple
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# state = population classes * fire classes
class FireManagementSpec:

    def __init__(self, population_classes=8, fire_classes=13, seed=1234, verbose=True):
        self.seed = seed
        self.verbose = verbose
        self.population_classes = population_classes
        self.fire_classes = fire_classes
        self.states = {}

        self.spec = mdpviz.MDPSpec()

        self._action_do_nothing = self.spec.action('do_nothing')
        self._action_burn = self.spec.action('burn')

        self._probabilities = {}
        self.name = f'fire_management_{population_classes}_{fire_classes}_{seed}'
        self.n_actions = 2
        self.n_states = self.fire_classes * self.population_classes

        self.reset()

    def reset(self):
        np.random.seed(self.seed)
        self._setup_mdp()

    def _reset_state_probabilities(self):
        self._probabilities = {}

    def _get_probability_for_state(self, pc, fc):
        state_name = self._get_state_name(pc, fc)
        if state_name not in self._probabilities:
            return None
        return self._probabilities[state_name]

    def _set_probability_for_state(self, pc, fc, p):
        state_name = self._get_state_name(pc, fc)
        if state_name not in self._probabilities:
            self._probabilities[state_name] = 0.
        self._probabilities[state_name] += p
        return self._probabilities[state_name]

    @staticmethod
    def _is_terminal(s):
        return False  # s == 0

    @staticmethod
    def get_habitat_suitability(years):
        if years < 0:
            msg = "Invalid years '%s', it should be positive." % str(years)
            raise ValueError(msg)
        if years <= 5:
            return 0.2 * years
        elif 5 <= years <= 10:
            return -0.1 * years + 1.5
        else:
            return 0.5

    @staticmethod
    def _get_state_name(pc, fc):
        return f'pc:{pc}, fc:{fc}'

    def _get_state(self, pc, fc):
        state_name = self._get_state_name(pc, fc)
        is_terminal = self._is_terminal(pc)
        if state_name not in self.states:
            state = self.spec.state(name=state_name, terminal_state=is_terminal)
            self.states[state_name] = state
        # print(f'{state_name} : {is_terminal}')
        state = self.states[state_name]
        return state

    def _add_state_transition_and_reward(self, pc, fc, action):
        cs = self._get_state(pc, fc)
        results = self._get_reward_and_new_state_values(pc, fc, action)
        for reward, npc, nfc, tp in results:
            ns = self._get_state(npc, nfc)
            ns = mdpviz.NextState(state=ns, weight=tp)
            self.spec.transition(state=cs, action=action, outcome=ns)
            self.spec.transition(state=cs, action=action, outcome=mdpviz.Reward(reward))
            if self.verbose:
                print(f'[state:action]: [{(pc, fc)} : {action.name}] -> new state: {(npc, nfc)}, '
                      f'p(t): {tp}, reward: {reward} ')

    def transition_fire_class(self, fc, action):
        if action == self._action_do_nothing:
            return (fc + 1) if fc < self.fire_classes - 1 else fc
        elif action == self._action_burn:
            return 0
        return fc

    def _get_reward_and_new_state_values(self, pc, fc, action, default_p=0.5):
        pop_change_down = -1
        pop_change_same = 0

        self._probabilities = {}
        transition_probability_up = None
#        if pc == 1 and fc == 0 and action == self._action_burn:
#            print()

        r = self.get_habitat_suitability(fc)
        fc = self.transition_fire_class(fc, action)
        if pc == 0:
            # dead
            return [[0.0, 0, fc, 1.0]]  # stays in same state
        if pc == self.population_classes - 1:
            pop_change_up = 0
            if action == self._action_burn:
                pop_change_same -= 1
                pop_change_down -= 1

            tsd = self._set_probability_for_state(pc=pc + pop_change_down,
                                                  fc=fc,
                                                  p=(1.0 - default_p) * (1.0 - r))
            tss = self._set_probability_for_state(pc=pc + pop_change_same,
                                                  fc=fc,
                                                  p=1.0 - tsd)
        else:
            # Population abundance class can stay the same, transition up, or
            # transition down.
            pop_change_same = 0
            pop_change_up = 1
            pop_change_down = -1

            # If action 1 is taken, then the patch is burned so the population
            # abundance moves down a class.
            if action == self._action_burn:
                pop_change_same -= 1
                pop_change_up -= 1
                pop_change_down -= (1 if pop_change_down > 0 else 0)

            tss = self._set_probability_for_state(pc=pc + pop_change_same,
                                                  fc=fc,
                                                  p=default_p)

            tsu = self._set_probability_for_state(pc=pc + pop_change_up,
                                                  fc=fc,
                                                  p=(1 - default_p)*r)
            # In the case when transition_down = 0 before the effect of an action
            # is applied, then the final state is going to be the same as that for
            # transition_same, so we need to add the probabilities together.
            tsd = self._set_probability_for_state(pc=pc + pop_change_down,
                                                  fc=fc,
                                                  p=(1 - default_p)*(1 - r))

        # build results
        results = []

        npc_up = pc + pop_change_up
        npc_down = pc + pop_change_down
        npc_same = pc + pop_change_same

        transition_probabilities = {
            (npc_up, self._get_probability_for_state(npc_up, fc)),
            (npc_down, self._get_probability_for_state(npc_down, fc)),
            (npc_same, self._get_probability_for_state(npc_same, fc))
        }

        for npc, probability in transition_probabilities:
            if probability is not None and probability > 0.0:
                reward = int(npc > 0)
                results.append((reward, npc, fc, probability))

        return results

    # noinspection PyStatementEffect
    def _setup_mdp(self):
        # build transitions
        for pc in range(0, self.population_classes):
            if self._is_terminal(pc):
                continue
            for fc in range(0, self.fire_classes):
                # actions
                self._add_state_transition_and_reward(pc=pc, fc=fc, action=self._action_do_nothing)
                self._add_state_transition_and_reward(pc=pc, fc=fc, action=self._action_burn)
                if self.verbose:
                    print()

    def get_transition_and_reward_arrays(self, p_default=0.5):
        return self.spec.get_transition_and_reward_arrays(p_default)

    def to_graph(self):
        return self.spec.to_graph()

    def to_env(self):
        return self.spec.to_discrete_env()
    
    def print_policy(self,policy):

        p = np.array(policy).reshape(self.population_classes, self.fire_classes)
        print("    " + " ".join("%2d" % f for f in range(self.fire_classes)))
        print("    " + "---" * self.fire_classes)
        for x in range(self.population_classes):
            print(" %2d|" % x + " ".join("%2d" % p[x, f] for f in
                                     range(self.fire_classes)))

In [5]:
fm_spec = FireManagementSpec()
envFM = fm_spec.to_env()

[state:action]: [(0, 0) : do_nothing] -> new state: (0, 1), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 0) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 1) : do_nothing] -> new state: (0, 2), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 1) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 2) : do_nothing] -> new state: (0, 3), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 2) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 3) : do_nothing] -> new state: (0, 4), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 3) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 4) : do_nothing] -> new state: (0, 5), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 4) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 5) : do_nothing] -> new state: (0, 6), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 5) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 6

In [6]:
fm_spec.reset()
fm_spec._setup_mdp()

[state:action]: [(0, 0) : do_nothing] -> new state: (0, 1), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 0) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 1) : do_nothing] -> new state: (0, 2), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 1) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 2) : do_nothing] -> new state: (0, 3), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 2) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 3) : do_nothing] -> new state: (0, 4), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 3) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 4) : do_nothing] -> new state: (0, 5), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 4) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 5) : do_nothing] -> new state: (0, 6), p(t): 1.0, reward: 0.0 
[state:action]: [(0, 5) : burn] -> new state: (0, 0), p(t): 1.0, reward: 0.0 

[state:action]: [(0, 6

In [7]:
# print the state space and action space
print(envFM.observation_space)
print(envFM.action_space)

Discrete(104)
Discrete(2)


In [8]:
P, R = fm_spec.get_transition_and_reward_arrays(p_default=0.5)

In [9]:
P

array([[[0.  , 1.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        ...,
        [0.  , 0.  , 0.  , ..., 0.75, 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.5 ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]],

       [[1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        ...,
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]]])

In [10]:
R

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.33333333, 0.        ],
       [0.4       , 0.2       ],
       [0.4       , 0.2       ],
       [0.6       , 0.4       ],
       [0.4       , 0.2       ],
       [0.6       , 0.4       ],
       [0.4       , 0.2       ],
       [0.6       , 0.4       ],
       [0.5       , 0.5       ],
       [0.5       , 0.25      ],
       [0.4       , 0.2       ],
       [0.6       , 0.4       ],
       [0.4       , 0.2       ],
       [0.6       , 0.4       ],
       [0.4       , 0.2       ],
       [0.6       , 0.4       ],
       [0.6       , 0.4       ],
       [0.

In [11]:
# For charting easier than going back to the top of notebook
discount_factor = .90

In [12]:
pi = mdp.PolicyIteration(P, R, discount_factor, policy0=None, max_iter=10**4, eval_type=0)

In [13]:
start = time()
statsPI = pi.run()
stop = time()
totalTime = stop-start
print('Time to train: ', totalTime)

Time to train:  0.0043408870697021484


In [14]:
statsPI

[{'State': None,
  'Action': None,
  'Reward': 5.735890878957754,
  'Error': 0.3645109751873985,
  'Time': 0.0024650096893310547,
  'V[0]': -5.574395986407413e-16,
  'Max V': 5.735890878957754,
  'Mean V': 4.3648078709727365,
  'Iteration': 1},
 {'State': None,
  'Action': None,
  'Reward': 5.8281435171116325,
  'Error': 0.1459245702190417,
  'Time': 0.0030498504638671875,
  'V[0]': 1.0965165675310187e-15,
  'Max V': 5.8281435171116325,
  'Mean V': 4.469407932521387,
  'Iteration': 2},
 {'State': None,
  'Action': None,
  'Reward': 5.902590343554508,
  'Error': 0.04378467377041595,
  'Time': 0.0034427642822265625,
  'V[0]': 6.0308411214206026e-15,
  'Max V': 5.902590343554508,
  'Mean V': 4.494835161119141,
  'Iteration': 3},
 {'State': None,
  'Action': None,
  'Reward': 5.9131787183625395,
  'Error': 1.7763568394002505e-15,
  'Time': 0.0038099288940429688,
  'V[0]': -6.0308411214206026e-15,
  'Max V': 5.9131787183625395,
  'Mean V': 4.499559537042314,
  'Iteration': 4},
 {'State': No

In [15]:
dfPI = pd.DataFrame(statsPI)
dfPI.to_csv('pi_small.csv')

In [16]:
fm_spec.print_policy(pi.policy)

     0  1  2  3  4  5  6  7  8  9 10 11 12
    ---------------------------------------
  0| 0  0  0  0  0  0  0  0  0  0  0  0  0
  1| 0  0  0  0  0  0  0  0  0  0  0  0  0
  2| 0  0  0  0  0  0  0  0  0  0  0  0  0
  3| 0  0  0  0  0  0  0  0  0  0  0  0  0
  4| 0  0  0  0  0  0  0  0  0  0  0  0  0
  5| 0  0  0  0  0  0  0  0  0  0  0  0  0
  6| 0  0  0  1  0  0  0  0  0  0  0  0  0
  7| 1  1  1  1  1  1  1  1  1  1  1  1  1


In [17]:
vi = mdp.ValueIteration(P, R, discount_factor, epsilon=0.01, max_iter=10**4, initial_value=0)

In [18]:
start = time()
statsVI = vi.run()
stop = time()
totalTime = stop-start
print('Time to train: ', totalTime)

Time to train:  0.0031354427337646484


In [19]:
statsVI

[{'State': None,
  'Action': None,
  'Reward': 0.6666666666666666,
  'Error': 0.6666666666666666,
  'Time': 0.00024819374084472656,
  'Max V': 0.6666666666666666,
  'Mean V': 0.48621794871794866,
  'Iteration': 1},
 {'State': None,
  'Action': None,
  'Reward': 1.2066666666666666,
  'Error': 0.6000000000000001,
  'Time': 0.0005800724029541016,
  'Max V': 1.2066666666666666,
  'Mean V': 0.9317035256410258,
  'Iteration': 2},
 {'State': None,
  'Action': None,
  'Reward': 1.6926666666666668,
  'Error': 0.54,
  'Time': 0.0006611347198486328,
  'Max V': 1.6926666666666668,
  'Mean V': 1.3221150881410257,
  'Iteration': 3},
 {'State': None,
  'Action': None,
  'Reward': 2.129337666666667,
  'Error': 0.4860000000000002,
  'Time': 0.000720977783203125,
  'Max V': 2.129337666666667,
  'Mean V': 1.6688554169871794,
  'Iteration': 4},
 {'State': None,
  'Action': None,
  'Reward': 2.520187371666667,
  'Error': 0.4345131600000003,
  'Time': 0.0009889602661132812,
  'Max V': 2.520187371666667,
  '

In [20]:
dfVI = pd.DataFrame(statsVI)
dfVI.to_csv('VI_small.csv')

In [21]:
vi.policy

(0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1)

In [22]:
fm_spec.print_policy(vi.policy)

     0  1  2  3  4  5  6  7  8  9 10 11 12
    ---------------------------------------
  0| 0  0  0  0  0  0  0  0  0  0  0  0  0
  1| 0  0  0  0  0  0  0  0  0  0  0  0  0
  2| 0  0  0  0  0  0  0  0  0  0  0  0  0
  3| 0  0  0  0  0  0  0  0  0  0  0  0  0
  4| 0  0  0  0  0  0  0  0  0  0  0  0  0
  5| 0  0  0  0  0  0  0  0  0  0  0  0  0
  6| 0  0  0  1  0  0  0  0  0  0  0  0  0
  7| 1  1  1  1  1  1  1  1  1  1  1  1  1


In [23]:
# Check converge

expected = pi.policy
all(expected[k] - vi.V[k] < 1e-12 for k in range(len(expected)))

True

In [24]:
#Check sizes
action_space_size = envFM.action_space.n
state_space_size = envFM.observation_space.n

In [25]:
print('Action space: ', action_space_size)
print('State space: ', state_space_size)

Action space:  2
State space:  104


In [26]:
Q = mdp.QLearning(P, R, discount_factor, alpha=0.30, alpha_decay=0.95, alpha_min=0.01,
                 epsilon=.30, epsilon_min=.01, epsilon_decay=0.01,
                 n_iter=10**4, skip_check=False, iter_callback=None)

In [27]:
start = time()
statsQ = Q.run()
stop = time()
totalTime = stop-start
print('Time to train: ', totalTime)

Time to train:  0.5794048309326172


In [28]:
fm_spec.print_policy(Q.policy)

     0  1  2  3  4  5  6  7  8  9 10 11 12
    ---------------------------------------
  0| 0  0  0  0  0  0  0  0  0  0  0  0  0
  1| 0  0  0  0  0  0  0  0  0  0  0  0  0
  2| 0  0  0  0  0  0  0  0  0  0  0  0  0
  3| 0  0  0  0  0  0  0  0  0  0  0  0  0
  4| 0  0  0  0  0  0  0  0  0  0  0  0  0
  5| 0  0  0  0  0  0  0  0  0  1  0  0  0
  6| 0  0  0  0  0  0  0  0  0  0  0  0  0
  7| 0  0  0  0  0  0  0  0  0  0  0  0  0


In [29]:
statsQ

[{'State': 47,
  'Action': 0,
  'Reward': 0.6,
  'Error': 0.18,
  'Time': 0.00011491775512695312,
  'Alpha': 0.3,
  'Epsilon': 0.3,
  'Gamma': 0.9,
  'V[0]': 0.0,
  'Max V': 0.18,
  'Mean V': 0.0017307692307692306,
  'Iteration': 1},
 {'State': 61,
  'Action': 0,
  'Reward': 0.6,
  'Error': 0.17099999999999999,
  'Time': 0.0005168914794921875,
  'Alpha': 0.285,
  'Epsilon': 0.01,
  'Gamma': 0.9,
  'V[0]': 0.0,
  'Max V': 0.18,
  'Mean V': 0.003375,
  'Iteration': 2},
 {'State': 75,
  'Action': 0,
  'Reward': 0.6,
  'Error': 0.16244999999999998,
  'Time': 0.0007688999176025391,
  'Alpha': 0.27075,
  'Epsilon': 0.01,
  'Gamma': 0.9,
  'V[0]': 0.0,
  'Max V': 0.18,
  'Mean V': 0.00493701923076923,
  'Iteration': 3},
 {'State': 62,
  'Action': 0,
  'Reward': 0.6,
  'Error': 0.19193325356249996,
  'Time': 0.0008728504180908203,
  'Alpha': 0.25721249999999996,
  'Epsilon': 0.01,
  'Gamma': 0.9,
  'V[0]': 0.0,
  'Max V': 0.19193325356249996,
  'Mean V': 0.006782531284254808,
  'Iteration': 4}

In [30]:
# Save for graphs
dfFMQ = pd.DataFrame(statsQ)
dfFMQ.to_csv('Q_small.csv')

In [31]:
print(Q)

P: 
array([[0.  , 1.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.75, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.5 ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

R: 
array([0., 0.])
array([0., 0.])

